In [ ]:
import anndata, math, pickle, os, fnmatch
import scanpy as sc
import numpy as np
import seaborn as sb
import matplotlib as mlp
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
from matplotlib.colors import ListedColormap
from matplotlib.ticker import MultipleLocator, LogLocator, LogFormatter, LogFormatterSciNotation, MaxNLocator
import matplotlib.patches as mpatches

In [ ]:
'''IMPORTING FUNCTIONS FROM GENERAL FUNCTIONS FILE'''

%run GeneralFunctions.ipynb

# Calculation Functions Figure 1

In [ ]:
# Finds the number of nuclei in a cluster for a sample 
    # sampleObj = AnnData object of the 
def findClustNum(sampleObj):
    # List where all of the nucleus counts will be held
    nucPerClust = []
    
    # Cluster number for sample list  
    clustLst = np.unique(list(sampleObj.obs['leiden_0.3']))
    
    print(clustLst)
    # Going through each cluster and finding the number of nuclei 
    for clust in clustLst:
        clustObj = sampleObj[sampleObj.obs['leiden_0.3'].isin([clust]),:]
        nucNum = clustObj.n_obs
        nucPerClust.append(nucNum)
    
    return(nucPerClust)    

In [ ]:
# Finds the percentile to use for max on the color bar in UMAPS
def percentileCalc(obj, gene):
    objGeneLst = list(obj.var_names)
    geneIdx = objGeneLst.index(gene)
    geneExpLst = obj.X.toarray()
    geneExp = [row[geneIdx] for row in geneExpLst]
    
    if (np.percentile(geneExp, 95)) > 0:
        return 'p95'
    elif (np.percentile(geneExp, 99)) > 0:
        return 'p99'
    else:
        return(0.1)

In [ ]:
# Averages the gene expression matrix across all cells (Helper Function for 'calculatingCorrelationMatrix')
    # dataObject = AnnData object you want to calculate the vector mean for 
    # clusterStr = List containing all of the clusters in the data object 
def calculatingVectorMeans(dataObject, clusterStr):
   
    # Creating AnnData Object for the chosen cluster from the chosen data
    clusterObj = dataObject[dataObject.obs['leiden_0.3'].isin([clusterStr]),:]
    # Turning the mean gene expression data for the cluster specified into a 2D list that can be averaged along rows 
    geneExpression = clusterObj.X.toarray()
    
    # Holds the mean of each col for all rows 
    vectorMean = []
    # Number of nuclei in the cluster specified (in dataObject.obs)
    nucleiCount = dataObject.n_obs
    # Number of genes looked at in the data (in dataObject.vars)
    geneCount = (len(geneExpression[0]))
    
    # Iterating through each column of the gene count array
    for i in range (geneCount):
        # Contains the sum for the same column in each row 
        geneSumPerCol = 0 
        # Iterating through each row of the gene count array 
        for vector in geneExpression:
            # Adding the same column per row 
            geneSumPerCol += vector[i]
        # Averaging the gene sums for the same column in all rows   
        aveGeneExp = geneSumPerCol / nucleiCount 
        # Adding the average for that column into the vectorMean list 
        vectorMean.append(aveGeneExp)
    
    return vectorMean

In [ ]:
# Creates the correlation matrix formatted for plotting a correlation heatmap (Helper Function for 'betweenSampleCorrelationCalculations')
    # clustLst_Half1 = List of the clusters found in the first half of the data 
    # data_Half1 = AnnData object of the first half of the data 
    # clustLst_Half2 = List of the clusters found in the second half of the data 
    # data_Half2 = AnnData object of the second half of the data 
def calculatingCorrelationMatrix(clustLst_Half1, data_Half1, clustLst_Half2, data_Half2):
    # sorting the list of cluster numbers for each sample 
    sortedClusters_Half1 = sortClust(clustLst_Half1)
    sortedClusters_Half2 = sortClust(clustLst_Half2)
    
    # Dictionary to hold vector means for each cluster 
    clusterMeans_Half1 = {}
    clusterMeans_Half2 = {}
    
    # Calculating the vector means for each cluster in Half 1
    for cluster_Half1 in sortedClusters_Half1:
        vectorMean_Half1 = calculatingVectorMeans(data_Half1, cluster_Half1)
        clusterMeans_Half1[cluster_Half1] = vectorMean_Half1
        
    # Calculating the vector means for each cluster in Half 2
    for cluster_Half2 in sortedClusters_Half2:
        vectorMean_Half2 = calculatingVectorMeans(data_Half2, cluster_Half2)
        clusterMeans_Half2[cluster_Half2] = vectorMean_Half2
    
    # where the matrix of the correlations is going to be held 
    corrMtrx = []
    
    # Looping through all of the clusters in sample half 1 
    for cluster_Half1 in clusterMeans_Half1: 
        row = []
        vectorMean_Half1 = clusterMeans_Half1[cluster_Half1]
        
        # Looping through all of the clusters in sample half 2 
        for cluster_Half2 in clusterMeans_Half2:
            vectorMean_Half2 = clusterMeans_Half2[cluster_Half2]
            # calculating the correlation of the two vectors 
            corrValue = np.corrcoef(vectorMean_Half1, vectorMean_Half2)[0, 1]
            # putting the correlation value into the list 
            row.append(corrValue)
        # putting the row of correlation values into the list 
        corrMtrx.append(row)

    return corrMtrx 

In [1]:
# Calculates the correlation heatmap matrix for a provided list of sample comaprisons you want to make and saves all 
# comparisons in a dictionary 
    # sampleComparisonLst = List containing strings of the comparisons you want to make 
        # Ex. [['sample1 and 2 condition', 'sample1', 'sample2'], ['sample3 and 4 condition', 'sample3', 'sample4'], etc...] 
def betweenSampleCorrelationCalculations(sampleComparisonLst):
    # Dictionary where the correlations for each sample will be held 
    corrMtx_dict = {}
    
    # Folder path for each sample half file 
    folderPath = f'F:/SampleData/SampleHalves/'
    
    # Looping through each sample comparison in the list 
    for sampleLst in sampleComparisonLst: 
        
        # Getting condition and name information for each sample 
        condition = sampleLst[0]
        sample1_name = sampleLst[1]

        # Getting folder paths for each half of sample 1
        sample1_half1_pathLst =  search_files(folderPath, f'{sample1_name}_Half1')
        sample1_half2_pathLst =  search_files(folderPath, f'{sample1_name}_Half2')

        # Reading AnnData object for each sample half 
        sample1_half1 = sc.read(sample1_half1_pathLst[0])
        sample1_half2 = sc.read(sample1_half2_pathLst[0])

        # List of cluster numbers for each sample half 
        sample1_half1_clustLst = list(np.unique(sample1_half1.obs['leiden_0.3']))
        sample1_half2_clustLst = list(np.unique(sample1_half2.obs['leiden_0.3']))

        # Getting information from sample 2 if we are doing between sample comparisons 
        if len(sampleLst) == 3: 
            # Getting sample name 
            sample2_name = sampleLst[2]
            
            # Getting folder paths for each half of sample 2
            sample2_half1_pathLst =  search_files(folderPath, f'{sample2_name}_Half1')
            sample2_half2_pathLst =  search_files(folderPath, f'{sample2_name}_Half2')
            
            # Reading AnnData object for each sample half 
            sample2_half1 = sc.read(sample2_half1_pathLst[0])
            sample2_half2 = sc.read(sample2_half2_pathLst[0])
            
            # List of cluster numbers for each sample half 
            sample2_half1_clustLst = list(np.unique(sample2_half1.obs['leiden_0.3']))
            sample2_half2_clustLst = list(np.unique(sample2_half2.obs['leiden_0.3']))
            
            # Calculating the correlation matrix for each sample 
            half1_corrMtx = calculatingCorrelationMatrix(sample1_half1_clustLst, sample1_half1, sample2_half1_clustLst, 
                                                         sample2_half1)
            half2_corrMtx = calculatingCorrelationMatrix(sample1_half2_clustLst, sample1_half2, sample2_half2_clustLst, 
                                                     sample2_half2)
            
            # Adding each sample half to the dictionary 
            corrMtx_dict[f'{condition}_{sample1_name}_{sample2_name}_Half1'] = half1_corrMtx
            corrMtx_dict[f'{condition}_{sample1_name}_{sample2_name}_Half2'] = half2_corrMtx
            
            print(sample1_name, sample2_name)
        
        else: 
            # Calculating correlation matrix for sample halves 
            corrMtx = calculatingCorrelationMatrix(sample1_half1_clustLst, sample1_half1, sample1_half2_clustLst, 
                                                   sample1_half2)
            
            # Adding correlation to dictionary 
            corrMtx_dict[sample1_name] = corrMtx
        
            print(sample1_name)
            
    return corrMtx_dict

# Plotting Functions Figure 1

In [ ]:
# PLOTTING FUNCTION: plots the best correlation between sample clusters
def dotPlotBestCorrel(correlMatrix, sample1_name, fontSize, ax):

    ylst = []
    pointLabel = []

    for correl in correlMatrix:
        maxNum = max(correl)
        numClst = correl.index(maxNum)
        ylst.append(maxNum)
        pointLabel.append(numClst)

    # Sample data
    x = [num for num in range (len(correlMatrix))]
    y = ylst
    
    # Making dots numbers (# https://stackoverflow.com/questions/69251212/matlplotlib-scatterplot-with-numbers-as-symbols-legend)
    ax.scatter(x, y, linestyle='None', color="white") 
    
    for i, txt in enumerate(pointLabel):
        ax.annotate(txt, (x[i], y[i]), ha="center", va="center")

    # Set the axis labels and title
    ax.set_xlabel(f'Clusters in Sample {sample1_name}', fontsize = 15)
    ax.set_ylabel('Correlation Value', fontsize = 15)
    ax.set_title('Highest Correl. Values btw Samples', fontsize = fontSize)
    ax.set_ylim(0, 1)
    ax.set_xticks(x)

In [3]:
# PLOTTING FUNCTION: plots best correlation vs. # of nuclei in cluster
def dotPlotNucNum(correlMatrix, sample1, sample2, fontSize, legendKey, ax):
    
    y1lst = []
    sample1_clustNum = np.unique(list(sample1.obs['leiden_0.3']))
    sample1_pointLabel = sortClust(sample1_clustNum)

    # Sample 1 data
    for correl in correlMatrix:
        maxNum = max(correl)
        y1lst.append(maxNum)
    
    x1 = findClustNum(sample1)
    y1 = y1lst

    # Sample 2 data
    
    y2lst = []
    sample2_clustNum = np.unique(list(sample2.obs['leiden_0.3']))
    sample2_pointLabel = sortClust(sample2_clustNum)
    
    for i in range(len(correlMatrix[0])):
        correlCol = [] 
        for correl in correlMatrix:
            correlCol.append(correl[i])
        maxNum = max(correlCol)
        y2lst.append(maxNum)
    
    x2 = findClustNum(sample2)
    y2 = y2lst
    
   # Scatter for Sample 1
    ax.scatter(x1, y1, linestyle='None', color="white", label='sample1') 
    
    for i, txt in enumerate(sample1_pointLabel):
        ax.annotate(txt, (x1[i], y1[i]), color='r', ha="center", va="center")
    
    # Scatter for Sample 2
    ax.scatter(x2, y2, linestyle='None', color="white") 
    
    for i, txt in enumerate(sample2_pointLabel):
        ax.annotate(txt, (x2[i], y2[i]), color='b', ha="center", va="center", label='sample2')

    # Set the axis labels and title
    ax.set_xlabel(f'Number of Nuclei', fontsize = 15)
    ax.set_ylabel('Correlation Value', fontsize = 15)
    ax.set_title('Highest Correl. vs Num. of Nuclei', fontsize = fontSize)
    ax.set_ylim(0, 1)
    
    # Creating custom legend
    if legendKey == 'halves':
        patchA = mpatches.Patch(color='r', label='Sample Half 1')
        patchB = mpatches.Patch(color='b', label='Sample Half 2')
    else:
        patchA = mpatches.Patch(color='r', label=legendKey[0])
        patchB = mpatches.Patch(color='b', label=legendKey[1])
    
    ax.legend(handles=[patchA, patchB], loc='lower right')